In [ ]:
# Libraries
import pandas as pd
import altair as alt
import numpy as np
import xgboost

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import metrics
from sklearn import tree


denver_dwelling = pd.read_csv("https://github.com/byuidatascience/data4dwellings/raw/master/data-raw/dwellings_ml/dwellings_ml.csv")
dwellings_ml = pd.read_csv("https://github.com/byuidatascience/data4dwellings/raw/master/data-raw/dwellings_ml/dwellings_ml.csv")

alt.data_transformers.enable('json')

## Grand Question 1 
Create 2-3 charts that evaluate potential relationships between the house variables and the variable before1980 Explain what you learn from the charts that could help a machine learning algorithm.



In [ ]:
# year column and something else
(
    alt.Chart(dwellings_ml)
    .mark_boxplot()
    .encode(
        y = alt.Y('yrbuilt:Q', 
        scale=alt.Scale(domain=[1800, 2020])
        ), 
        x = alt.X('arcstyle_ONE AND HALF-STORY:O')
        # , color = "before1980:O"
        )
)

In [ ]:
# year column and something else

(
    alt.Chart(dwellings_ml)
    .mark_boxplot()
    .encode(
        y = alt.Y('yrbuilt:Q', 
        scale=alt.Scale(domain=[1850, 2020])
        ), # zoom into the revelant years
        x = alt.X('arcstyle_SPLIT LEVEL:O')
        # , color = "before1980:O"
        )
)

In [ ]:
(
    alt.Chart(dwellings_ml)
    .mark_boxplot()
    .encode(
        y = alt.Y('yrbuilt:Q', 
        scale=alt.Scale(domain=[1850, 2020])
        ), # zoom into the revelant years
        x = alt.X('condition_AVG:O')
        # , color = "before1980:O"
        )
)

## Grand Question 2
Build a classification model labeling houses as being built “before 1980” or “during or after 1980”. Your goal is to reach 90% accuracy. Explain your final model choice (algorithm, tuning parameters, etc) and describe what other models you tried.




In [ ]:
# "X" or "independent"
features = dwellings_ml.drop(columns = ['before1980', 'yrbuilt','parcel'])

# "y" or "dependent" or "outcome"
targets = dwellings_ml.before1980
# targets must already be boolean (classification type)


# split the data!
x_train, x_test, y_train, y_test = train_test_split(features, targets, test_size = .20, random_state = 76)



In [ ]:
classifier_DT = RandomForestClassifier(max_depth=30) #create a classification model
 # I fear that this is over fitting. How can I check for this?
# classifier_DT = DecisionClassifier(max_depth=10)

# train the model
classifier_DT.fit(x_train, y_train)

# use your model to make predictions!
y_predicted = classifier_DT.predict(x_test)


# test how accurate those predictions are
metrics.accuracy_score(y_test, y_predicted)

I opted to do a random forest model. I have the training data set at 80% and the test data at 20%. The max depth of the model is at 10 which gives us 90% accuracy. 

## Grand Question 3 
Justify your classification model by discussing the most important features selected by your model. This discussion should include a chart and a description of the features.



In [ ]:
feature_df = pd.DataFrame({'features':features.columns, 'importance':classifier_DT.feature_importances_})

feature_importance = feature_df.sort_values('importance', ascending = False).reset_index().head(10)

feature_importance

For the features in our model (xs), we use everything except the: `before1980`, `yrbuilt`, and `parcel` columns. Our target (Y variable), we use `before1980`. 

We can find which are the most contributing features by looking at the chart below.

In [ ]:
feat_importances = pd.Series(
    classifier_DT
    .feature_importances_, index=x_test
    .columns
)

significance_graph = (
    feat_importances
    .nlargest(10)
    .plot(kind='barh')
)
significance_graph

## Grand Question 4
Describe the quality of your classification model using 2-3 different evaluation metrics. You also need to explain how to interpret each of the evaluation metrics you use.



In [ ]:
# Accuracy Score 

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, classifier_DT.predict(x_test))
accuracy

Accuracy is the correct answers divided by the correct answers and incorrect answers. This is similiar to a grade on a test. 

In [ ]:
# Recall Score
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_predicted, average='binary')
recall

Recall is the number of true positives divided by the number of true positives plus the number of false negatives. This metric's nature is all about the *actual positives*. 

In [ ]:
# Precision Score

from sklearn.metrics import precision_score
precision = precision_score(y_test, y_predicted, average='binary')
precision

Precision is the number of true positives divided by the number of true positives plus false positives. This metric's nature is all about the *predicted positives*. 